# Example: Get, inspect, change, and put back a methodset method

Here outlines how to interact with methodset methods and instrument methods.

- Finding and getting a methodset method.
- Inspecting what types of instrument methods are in the methodset method.
- Inspecting the instrument parameters in the method, including column temperature and gradient table.
- Changing instrument parameters and posting the new method.

# Finding and getting a methodset method

We first find the list of all methodset methods, select one, and get that from Empower:

In [1]:
import pprint

from OptiHPLCHandler import EmpowerHandler

# Create an instance of the EmpowerHandler class
handler = EmpowerHandler(project="WebAPI_test", address="http://***REMOVED***:3076/")

# Get the list of methods, select one, and get the method details
with handler:
    method_list = handler.GetMethodList() # Get the list of methodset methods
    method_name = method_list[0] # Select the first method
    print(method_name)
    full_method = handler.GetMethodsetMethod(method_name)

C:\Users\srfu\MATLAB\Projects\OptiHPLCHandler\src\OptiHPLCHandler\empower_api_core.py:172: UserWarning: The password will be sent in plain text.
  warnings.warn("The password will be sent in plain text.")


HTTPError: 400 Client Error: Bad Request for url: http://***REMOVED***:3076/authentication/login

# Inspecting the methodset method

We can now look into the instrument methods in the methodset methods. 

In [2]:
pp = pprint.PrettyPrinter(indent=2)

print(full_method)
print(f"Valve positions: {full_method.instrument_method_list[-1].valve_position}") # Printing the valve position for the solvent manager instrument method in the list
print("\n\nStart of gradient table:\n")
pp.pprint(full_method.gradient_table[0:2]) # Printing the first two entries gradient table. Notice that we do not need to know the tag names to print the gradient table
print("\n\nDetector settings:")
print(full_method.instrument_method_list[0]["ChannelA"]) # Printing the settings for a single channel where we know the tag name (ChannelA)

EmpowerMethodSetMethod with 4 instrument methods of types EmpowerInstrumentMethod, EmpowerInstrumentMethod, EmpowerInstrumentMethod, BSMMethod
Valve positions: ['A2', 'B2']


Start of gradient table:

[ { 'CompositionA': '95.0',
    'CompositionB': '5.0',
    'Curve': 'Initial',
    'Flow': '0.900',
    'Time': 'Initial'},
  { 'CompositionA': '20.0',
    'CompositionB': '80.0',
    'Curve': '6',
    'Flow': '0.900',
    'Time': '2.00'}]


Detector settings:

    <Description />
    <Wavelength>280</Wavelength>
    <DataRate>SingleDataRate_20A</DataRate>
    <DataMode>SingleMode_1A</DataMode>
    <FilterType>Filter_2</FilterType>
    <TimeConstant>0.1</TimeConstant>
    <RatioMinimum>0.0001</RatioMinimum>
    <AutoZeroWavelength>Az_3</AutoZeroWavelength>
    <AutoZeroInjectStart>true</AutoZeroInjectStart>
    <AutoZeroEventOrKey>true</AutoZeroEventOrKey>
  


Note that most of the methods are simply EmpowerInstrumentMethods. That is the generic
type for instrument methods that aren't specifically accounted for. At the moment, that
is every type of method expect:
 - Solvent manager methods (BSMMethod and QSMMethod), which have the special properties valve_position and gradient_table
 - Column oven methods (SampleManagerMethod), which have the special property column_temperature.

All of the special properties can be accessed as set from the `EmpowerMethodSetMethod`.
Getting `EmpowerMethodSetMethod.column_temperature` will produce an error if the
methodset method controls several column ovens (e.g. one in a sample manager and one in
a column manager), and the column ovens have different temperatures. Setting 
`EmpowerMethodSetMethod.column_temperature` will set the temperature for all column
ovens there are any, and produce an error otherwise.

# Changing instrument parameters and posting the new method.

We can also change the values of the methodset method, and post the changed method back
to Empower, so we can use it to analyse samples.

Remember to give the method a new name before posting it to Empower. OptiHPLCHandler
does not allow changing methods in Empower, only creating new ones.

In [4]:
gradient_table = full_method.gradient_table # Get the gradient table
for step in gradient_table:
    step["Flow"] = 0.4 # Set the flow to 0.5 mL/min for all steps
full_method.gradient_table = gradient_table # Set the gradient table to the updated gradient table
full_method.valve_position = ["A2", "B1"] # Set the valve position to A2 and B1. You can also set only one of the valves.
full_method.method_name ="New method name" # Set the method name
with handler:
    handler.PostMethodsetMethod(full_method) # Post the updated method to Empower